In [1]:
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [2]:
previsores = pd.read_csv('entradas_breast.csv')
previsores.head(2)

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave_points_mean,symmetry_mean,fractal_dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave_points_worst,symmetry_worst,fractal_dimension_worst
0,17.99,10.38,122.8,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.9,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.8,1956.0,0.1238,0.1866,0.2416,186.0000,275.0000,0.08902


In [3]:
classe = pd.read_csv('saidas_breast.csv')
classe.head(2)

,0
0,0
1,0


In [12]:
import gc
from tensorflow.keras import backend as k
from tensorflow.keras.callbacks import Callback

In [13]:
class ClearMemory(Callback):
    def on_epoch_end(self, epoch, logs=None):
        gc.collect()
        k.clear_session()

In [14]:
def criarRede(optimizer, loos, kernel_initializer, activation, neurons):
    # cleanup
    #K.clear_session()
    
    classificador = Sequential()
    
    classificador.add(Dense(units = neurons, activation = activation, 
                        kernel_initializer = kernel_initializer, input_dim = 30))
    
    classificador.add(Dropout(0.2))
    
    classificador.add(Dense(units = neurons, activation = activation, 
                        kernel_initializer = kernel_initializer))
    
    classificador.add(Dropout(0.2))
    
    classificador.add(Dense(units = 1, activation = 'sigmoid'))
    
    classificador.compile(optimizer = optimizer, 
                          loss = loos,
                          metrics = ['binary_accuracy'],
                          run_eagerly=True)
    
    return classificador

In [15]:
classificador = KerasClassifier(build_fn = criarRede)
parametros = {'batch_size': [10, 30],
              'epochs': [50, 100],
              'callbacks': [ClearMemory()],
              'optimizer': ['adam', 'sgd'],
              'loos': ['binary_crossentropy', 'hinge'],
              'kernel_initializer': ['random_uniform', 'normal'],
              'activation': ['relu', 'tanh'],
              'neurons': [16, 8]}

In [16]:
grid_search = GridSearchCV(estimator = classificador,
                           param_grid = parametros,
                           scoring = 'accuracy',
                           cv = 5)

In [17]:
import time

In [ ]:
    start = time.time()
    grid_search = grid_search.fit(previsores, classe)

Epoch 1/50
46/46 [==============================] - 1s 14ms/step - loss: 1.0723 - binary_accuracy: 0.5538
Epoch 2/50
46/46 [==============================] - 1s 14ms/step - loss: 0.6281 - binary_accuracy: 0.6286
Epoch 3/50
46/46 [==============================] - 1s 14ms/step - loss: 0.6289 - binary_accuracy: 0.6659
Epoch 4/50
46/46 [==============================] - 1s 14ms/step - loss: 0.5280 - binary_accuracy: 0.7473
Epoch 5/50
46/46 [==============================] - 1s 14ms/step - loss: 0.5068 - binary_accuracy: 0.7231
Epoch 6/50
46/46 [==============================] - 1s 14ms/step - loss: 0.5091 - binary_accuracy: 0.7297
Epoch 7/50
46/46 [==============================] - 1s 14ms/step - loss: 0.4777 - binary_accuracy: 0.7802
Epoch 8/50
46/46 [==============================] - 1s 14ms/step - loss: 0.4635 - binary_accuracy: 0.8264
Epoch 9/50
46/46 [==============================] - 1s 14ms/step - loss: 0.4197 - binary_accuracy: 0.8066
Epoch 10/50
46/46 [===========================

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



16/16 [==============================] - 0s 16ms/step - loss: 0.6371 - binary_accuracy: 0.6835
Epoch 15/100
16/16 [==============================] - 0s 17ms/step - loss: 0.6362 - binary_accuracy: 0.6901
Epoch 16/100
16/16 [==============================] - 0s 16ms/step - loss: 0.6470 - binary_accuracy: 0.6813
Epoch 17/100
16/16 [==============================] - 0s 16ms/step - loss: 0.6498 - binary_accuracy: 0.6791
Epoch 18/100
16/16 [==============================] - 0s 16ms/step - loss: 0.6291 - binary_accuracy: 0.6835
Epoch 19/100
16/16 [==============================] - 0s 16ms/step - loss: 0.6384 - binary_accuracy: 0.6857
Epoch 20/100
16/16 [==============================] - 0s 16ms/step - loss: 0.6513 - binary_accuracy: 0.6835
Epoch 21/100
16/16 [==============================] - 0s 16ms/step - loss: 0.6329 - binary_accuracy: 0.6835
Epoch 22/100
16/16 [==============================] - 0s 16ms/step - loss: 0.6358 - binary_accuracy: 0.6857
Epoch 23/100
16/16 [=====================

In [ ]:
end = time.time()
print(end - start)

In [12]:
melhores_parametros = grid_search.best_params_
melhores_parametros

{'activation': 'relu',
 'batch_size': 10,
 'epochs': 50,
 'kernel_initializer': 'random_uniform',
 'loos': 'binary_crossentropy',
 'neurons': 8,
 'optimizer': 'adam'}

In [13]:
melhor_precisao = grid_search.best_score_
melhor_precisao

0.8963514982145628